In [1]:
%%capture
!pip install -U crowd-kit==1.0.0

In [2]:
from crowdkit.datasets import load_dataset

In [3]:
df, df_gt = load_dataset('relevance-2')

In [4]:
df

,worker,task,label
0,w851,t30685,1
1,w6991,t30008,0
2,w2596,t36316,0
3,w5507,t15145,1
4,w2982,t44785,1
...,...,...,...
475531,w4660,t62250,1
475532,w6630,t46626,0
475533,w4605,t93513,1
475534,w1928,t29002,0


In [5]:
df_gt

task
t30006    0
t33578    0
t22462    1
t52093    0
t26935    0
         ..
t57345    1
t81052    1
t7189     1
t80463    0
t93643    0
Name: label, Length: 10079, dtype: int64

In [6]:
from crowdkit.aggregation import MajorityVote, Wawa, DawidSkene

In [7]:
agg_mv = MajorityVote().fit_predict(df)
agg_mv

task
t0       1
t1       1
t10      1
t100     0
t1000    0
        ..
t9995    1
t9996    0
t9997    0
t9998    0
t9999    1
Name: agg_label, Length: 99319, dtype: int64

In [8]:
agg_wawa = Wawa().fit_predict(df)
agg_wawa

task
t0       1
t1       1
t10      1
t100     0
t1000    0
        ..
t9995    1
t9996    0
t9997    0
t9998    0
t9999    1
Name: agg_label, Length: 99319, dtype: int64

In [9]:
agg_ds = DawidSkene(n_iter=10).fit_predict(df)
agg_ds

task
t30685    1
t30008    0
t36316    0
t15145    1
t44785    0
         ..
t95222    0
t83525    0
t49227    0
t96106    1
t16185    1
Name: agg_label, Length: 99319, dtype: int64

In [10]:
from sklearn.metrics import f1_score

In [11]:
f1_score(df_gt, agg_mv[df_gt.index])

0.7621861152141802

In [12]:
f1_score(df_gt, agg_wawa[df_gt.index])

0.7610675039246467

In [13]:
f1_score(df_gt, agg_ds[df_gt.index])

0.7883563142919676

In [14]:
agg_ds.to_frame('label').reset_index()

,task,label
0,t30685,1
1,t30008,0
2,t36316,0
3,t15145,1
4,t44785,0
...,...,...
99314,t95222,0
99315,t83525,0
99316,t49227,0
99317,t96106,1
